In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from tqdm import tqdm
import numpy as np
import random

# ============================
# 1. Setup and Hyperparameters
# ============================

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # Ensures deterministic behavior, may slow down training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
num_epochs = 30
batch_size = 4096  # Large batch size
learning_rate = 0.1
momentum = 0.9
weight_decay = 5e-4
random_percentage = 0.1  # Percentage of parameters to reinitialize (e.g., 0.1 for 10%)

# PGD attack parameters for testing
pgd_steps = 7
pgd_alpha = 2/255
pgd_eps = 8/255

# ============================
# 2. Data Loading and Preprocessing
# ============================

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 training and test datasets
print("Loading CIFAR-10 dataset...")
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

# Convert datasets to tensors and move to GPU
print("Converting training data to tensors and moving to GPU...")
train_inputs = torch.stack([trainset[i][0] for i in range(len(trainset))]).to(device)
train_targets = torch.tensor([trainset[i][1] for i in range(len(trainset))]).to(device)

print("Converting test data to tensors and moving to GPU...")
test_inputs = torch.stack([testset[i][0] for i in range(len(testset))]).to(device)
test_targets = torch.tensor([testset[i][1] for i in range(len(testset))]).to(device)

print("Data loading complete.")

# ============================
# 3. Model Definition
# ============================

class ResNet18_CIFAR10(nn.Module):
    def __init__(self):
        super(ResNet18_CIFAR10, self).__init__()
        self.model = resnet18(pretrained=False, num_classes=10)
        # Modify the first convolution layer for CIFAR-10
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
    
    def forward(self, x):
        return self.model(x)

model = ResNet18_CIFAR10().to(device)

# ============================
# 4. Optimizer and Scheduler
# ============================

optimizer = optim.SGD(model.parameters(), lr=learning_rate, 
                      momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25], gamma=0.1)

# Define loss function
criterion = nn.CrossEntropyLoss()

# ============================
# 5. Random Reinitialization Function
# ============================

def random_reinitialize_parameters(model, percentage):
    """
    Randomly reinitialize a percentage of the model's parameters (excluding final layers).
    Returns a list of tuples containing parameter references and their original values.
    """
    params_to_reinit = []
    original_values = []
    
    # Collect all parameters except the final layer
    parameters = []
    for name, param in model.named_parameters():
        if 'fc' not in name:  # Exclude final fully connected layer
            parameters.append((name, param))
    
    num_params = len(parameters)
    num_to_reinit = int(num_params * percentage)
    selected_indices = random.sample(range(num_params), num_to_reinit)
    
    for idx in selected_indices:
        name, param = parameters[idx]
        # Save original values
        original_values.append((param, param.data.clone()))
        # Reinitialize parameter
        if param.dim() > 1:
            nn.init.kaiming_normal_(param.data)
        else:
            nn.init.zeros_(param.data)
        # Freeze parameter
        param.requires_grad = False
    
    return original_values

def restore_parameters(original_values):
    """
    Restore the original values of parameters and unfreeze them.
    """
    for param, original_value in original_values:
        param.data.copy_(original_value)
        param.requires_grad = True

# ============================
# 6. PGD Attack Implementation for Testing
# ============================

def pgd_attack(model, images, labels, eps=8/255, alpha=2/255, iters=7):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    ori_images = images.clone().detach()

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad = images.grad.data

        # Update adversarial images
        images = images + alpha * torch.sign(grad)
        # Clamp perturbations
        perturbation = torch.clamp(images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + perturbation, min=0, max=1).detach()

    return images

# ============================
# 7. Training and Evaluation Functions
# ============================

def train_random_reinit(model, train_inputs, train_targets, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Shuffle the data at the beginning of each epoch
    perm = torch.randperm(train_inputs.size(0))
    shuffled_inputs = train_inputs[perm]
    shuffled_targets = train_targets[perm]

    num_batches = train_inputs.size(0) // batch_size

    loop = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx in loop:
        # Create batch
        start = batch_idx * batch_size
        end = start + batch_size
        inputs = shuffled_inputs[start:end]
        targets = shuffled_targets[start:end]

        # Randomly reinitialize parameters and freeze them
        original_values = random_reinitialize_parameters(model, random_percentage)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Restore parameters and unfreeze them
        restore_parameters(original_values)

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Update progress bar
        loop.set_postfix(loss=running_loss/(batch_idx+1), acc=100.*correct/total)

def evaluate_random_reinit(model, test_inputs, test_targets):
    model.eval()
    correct_clean = 0
    total_clean = 0
    correct_adv = 0
    total_adv = 0

    with torch.no_grad():
        # Clean accuracy
        outputs = model(test_inputs)
        _, predicted = outputs.max(1)
        total_clean += test_targets.size(0)
        correct_clean += predicted.eq(test_targets).sum().item()

    # Evaluate adversarial accuracy
    for i in tqdm(range(0, test_inputs.size(0), batch_size), desc="Evaluating Adversarial"):
        start = i
        end = min(i + batch_size, test_inputs.size(0))
        inputs = test_inputs[start:end]
        targets = test_targets[start:end]

        # Generate adversarial examples
        adv_inputs = pgd_attack(model, inputs, targets, eps=pgd_eps, alpha=pgd_alpha, iters=pgd_steps)

        # Predict on adversarial examples
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        total_adv += targets.size(0)
        correct_adv += predicted.eq(targets).sum().item()

    clean_acc = 100. * correct_clean / total_clean
    adv_acc = 100. * correct_adv / total_adv

    print(f"\nClean Accuracy: {clean_acc:.2f}%")
    print(f"Adversarial Accuracy: {adv_acc:.2f}%\n")

    return clean_acc, adv_acc

# ============================
# 8. Training Loop
# ============================

best_clean_acc = 0.0
best_adv_acc = 0.0

for epoch in range(num_epochs):
    train_random_reinit(model, train_inputs, train_targets, optimizer, epoch)
    clean_acc, adv_acc = evaluate_random_reinit(model, test_inputs, test_targets)
    scheduler.step()

    # Save the best model based on adversarial accuracy
    if adv_acc > best_adv_acc:
        best_adv_acc = adv_acc
        torch.save(model.state_dict(), "resnet18_cifar10_random_reinit_best.pth")
        print(f"Best model saved with adversarial accuracy: {best_adv_acc:.2f}%")

print("Training complete.")
print(f"Best Clean Accuracy: {clean_acc:.2f}%")
print(f"Best Adversarial Accuracy: {best_adv_acc:.2f}%")


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from tqdm import tqdm
import numpy as np
import random

# ============================
# 1. Setup and Hyperparameters
# ============================

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # Ensures deterministic behavior, may slow down training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
num_epochs = 30
batch_size = 4096  # Adjust based on your GPU memory
learning_rate = 0.1
momentum = 0.9
weight_decay = 5e-4
random_percentage = 0.01  # Percentage of parameters to reinitialize (e.g., 0.1 for 10%)

# PGD attack parameters for training and testing
pgd_steps = 7
pgd_alpha = 2/255
pgd_eps = 8/255

# ============================
# 2. Data Loading and Preprocessing
# ============================

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 training and test datasets
print("Loading CIFAR-10 dataset...")
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

# Convert datasets to tensors and move to GPU
print("Converting training data to tensors and moving to GPU...")
train_inputs = torch.stack([trainset[i][0] for i in range(len(trainset))]).to(device)
train_targets = torch.tensor([trainset[i][1] for i in range(len(trainset))]).to(device)

print("Converting test data to tensors and moving to GPU...")
test_inputs = torch.stack([testset[i][0] for i in range(len(testset))]).to(device)
test_targets = torch.tensor([testset[i][1] for i in range(len(testset))]).to(device)

print("Data loading complete.")

# ============================
# 3. Model Definition
# ============================

class ResNet18_CIFAR10(nn.Module):
    def __init__(self):
        super(ResNet18_CIFAR10, self).__init__()
        self.model = resnet18(pretrained=False, num_classes=10)
        # Modify the first convolution layer for CIFAR-10
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
    
    def forward(self, x):
        return self.model(x)

model = ResNet18_CIFAR10().to(device)

# ============================
# 4. Optimizer and Scheduler
# ============================

optimizer = optim.SGD(model.parameters(), lr=learning_rate, 
                      momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25], gamma=0.1)

# Define loss function
criterion = nn.CrossEntropyLoss()

# ============================
# 5. Random Reinitialization Function
# ============================

def random_reinitialize_parameters(model, percentage):
    """
    Randomly reinitialize a percentage of the model's parameters (excluding final layers).
    Returns a list of tuples containing parameter references and their original values.
    """
    params_to_reinit = []
    original_values = []
    
    # Collect all parameters except the final layer
    parameters = []
    for name, param in model.named_parameters():
        if 'fc' not in name and 'classifier' not in name:  # Adjust based on model's final layer name
            parameters.append((name, param))
    
    num_params = len(parameters)
    num_to_reinit = int(num_params * percentage)
    selected_indices = random.sample(range(num_params), num_to_reinit)
    
    for idx in selected_indices:
        name, param = parameters[idx]
        # Save original values
        original_values.append((param, param.data.clone()))
        # Reinitialize parameter
        if param.dim() > 1:
            nn.init.kaiming_normal_(param.data)
        else:
            nn.init.zeros_(param.data)
        # Freeze parameter
        param.requires_grad = False
    
    return original_values

def restore_parameters(original_values):
    """
    Restore the original values of parameters and unfreeze them.
    """
    for param, original_value in original_values:
        param.data.copy_(original_value)
        param.requires_grad = True

# ============================
# 6. PGD Attack Implementation
# ============================

def pgd_attack(model, images, labels, eps=8/255, alpha=2/255, iters=7):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    ori_images = images.clone().detach()

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad = images.grad.data

        # Update adversarial images
        images = images + alpha * torch.sign(grad)
        # Clamp perturbations
        perturbation = torch.clamp(images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + perturbation, min=0, max=1).detach()

    return images

# ============================
# 7. Training and Evaluation Functions
# ============================

def train_adversarial_random_reinit(model, train_inputs, train_targets, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Shuffle the data at the beginning of each epoch
    perm = torch.randperm(train_inputs.size(0))
    shuffled_inputs = train_inputs[perm]
    shuffled_targets = train_targets[perm]

    num_batches = train_inputs.size(0) // batch_size

    loop = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx in loop:
        # Create batch
        start = batch_idx * batch_size
        end = start + batch_size
        inputs = shuffled_inputs[start:end]
        targets = shuffled_targets[start:end]

        # Generate adversarial examples
        adv_inputs = pgd_attack(model, inputs, targets, eps=pgd_eps, alpha=pgd_alpha, iters=pgd_steps)

        # Randomly reinitialize parameters and freeze them
        original_values = random_reinitialize_parameters(model, random_percentage)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass on adversarial examples
        outputs = model(adv_inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Restore parameters and unfreeze them
        restore_parameters(original_values)

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Update progress bar
        loop.set_postfix(loss=running_loss/(batch_idx+1), acc=100.*correct/total)

def evaluate_adversarial_random_reinit(model, test_inputs, test_targets):
    model.eval()
    correct_clean = 0
    total_clean = 0
    correct_adv = 0
    total_adv = 0

    with torch.no_grad():
        # Clean accuracy
        outputs = model(test_inputs)
        _, predicted = outputs.max(1)
        total_clean += test_targets.size(0)
        correct_clean += predicted.eq(test_targets).sum().item()

    # Evaluate adversarial accuracy
    for i in tqdm(range(0, test_inputs.size(0), batch_size), desc="Evaluating Adversarial"):
        start = i
        end = min(i + batch_size, test_inputs.size(0))
        inputs = test_inputs[start:end]
        targets = test_targets[start:end]

        # Generate adversarial examples
        adv_inputs = pgd_attack(model, inputs, targets, eps=pgd_eps, alpha=pgd_alpha, iters=pgd_steps)

        # Predict on adversarial examples
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        total_adv += targets.size(0)
        correct_adv += predicted.eq(targets).sum().item()

    clean_acc = 100. * correct_clean / total_clean
    adv_acc = 100. * correct_adv / total_adv

    print(f"\nClean Accuracy: {clean_acc:.2f}%")
    print(f"Adversarial Accuracy: {adv_acc:.2f}%\n")

    return clean_acc, adv_acc

# ============================
# 8. Training Loop
# ============================

best_adv_acc = 0.0
best_clean_acc = 0.0

for epoch in range(num_epochs):
    train_adversarial_random_reinit(model, train_inputs, train_targets, optimizer, epoch)
    clean_acc, adv_acc = evaluate_adversarial_random_reinit(model, test_inputs, test_targets)
    scheduler.step()

    # Save the best model based on adversarial accuracy
    if adv_acc > best_adv_acc:
        best_adv_acc = adv_acc
        torch.save(model.state_dict(), "resnet18_cifar10_adversarial_random_reinit_best.pth")
        print(f"Best model saved with adversarial accuracy: {best_adv_acc:.2f}%")

    # Optionally, also save based on clean accuracy
    if clean_acc > best_clean_acc:
        best_clean_acc = clean_acc
        torch.save(model.state_dict(), "resnet18_cifar10_adversarial_random_reinit_best_clean.pth")
        print(f"Best model saved with clean accuracy: {best_clean_acc:.2f}%")

print("Training complete.")
print(f"Best Clean Accuracy: {best_clean_acc:.2f}%")
print(f"Best Adversarial Accuracy: {best_adv_acc:.2f}%")


Using device: cuda
Loading CIFAR-10 dataset...
Files already downloaded and verified
Files already downloaded and verified
Converting training data to tensors and moving to GPU...
Converting test data to tensors and moving to GPU...
Data loading complete.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating Adversarial: 100%|██████████| 3/3 [00:10<00:00,  3.65s/it]



Clean Accuracy: 9.21%
Adversarial Accuracy: 8.16%

Best model saved with adversarial accuracy: 8.16%
Best model saved with clean accuracy: 9.21%


Evaluating Adversarial: 100%|██████████| 3/3 [00:10<00:00,  3.65s/it]



Clean Accuracy: 9.25%
Adversarial Accuracy: 10.65%

Best model saved with adversarial accuracy: 10.65%
Best model saved with clean accuracy: 9.25%


...


Clean Accuracy: 22.83%
Adversarial Accuracy: 21.80%



Evaluating Adversarial: 100%|██████████| 3/3 [00:10<00:00,  3.65s/it]



Clean Accuracy: 22.68%
Adversarial Accuracy: 21.67%



Evaluating Adversarial: 100%|██████████| 3/3 [00:10<00:00,  3.65s/it]


Clean Accuracy: 22.67%
Adversarial Accuracy: 21.66%

Training complete.
Best Clean Accuracy: 27.32%
Best Adversarial Accuracy: 23.98%
